In [ ]:
import requests
import tarfile
import time


# URL du fichier à télécharger
url = 'https://echanges.dila.gouv.fr/OPENDATA/Debats/AN/2022/AN_2022001.taz'

# Nom du fichier local pour enregistrer le contenu
nom_fichier = '../data/raw/AN_2022001.taz'


# Effectuer la requête GET pour récupérer le fichier
time_start = time.time()

response = requests.get(url)
time_townload = time.time() - time_start
print(f"Téléchargement effectué en {time_townload} secondes.")
# Vérifier si la requête a réussi (code 200)
if response.status_code == 200:
    print(f"Le fichier a été téléchargé avec succès sous le nom : {nom_fichier}")
    with open(nom_fichier, 'wb') as f:
        f.write(response.content)
    print(f"time to write: {time.time() - time_start:.2f} secondes.")

else:
    print(f"Échec du téléchargement. Code de statut : {response.status_code}")


Téléchargement effectué en 1.73 secondes.
Le fichier a été téléchargé avec succès sous le nom : ../data/raw/AN_2022001.taz
time to write: 1.73 secondes.


In [ ]:
fichier_tar = '../data/raw/AN_2022001.taz'

with tarfile.open(nom_fichier, "r:*") as tar:
    # Extraire tout le contenu dans le dossier courant
    tar.extractall(path='')


In [2]:
import io
import tarfile
import xml.etree.ElementTree as ET

nom_fichier_taz = "../data/raw/AN_2022001.taz"

# Ouvrir le .taz (tar.gz)
with tarfile.open(nom_fichier_taz, "r:*") as taz:
    # Trouver le .tar à l'intérieur
    membres_tar = [m for m in taz.getmembers() if m.name.endswith(".tar")]
    if not membres_tar:
        raise ValueError("Aucun fichier .tar trouvé dans le .taz.")
    
    membre_tar = membres_tar[0]
    print(f"Ouverture du fichier TAR interne : {membre_tar.name}")

    # Lire le .tar interne en mémoire
    tar_bytes = taz.extractfile(membre_tar).read()
    tar_buffer = io.BytesIO(tar_bytes)

    # Ouvrir le .tar interne depuis la mémoire
    with tarfile.open(fileobj=tar_buffer, mode="r:") as tar:
        # ⚠️ bien utiliser getmembers() (avec un 's')
        for membre in tar.getmembers():
            if membre.name.endswith(".xml"):
                print(f"Lecture du XML : {membre.name}")

                f = tar.extractfile(membre)
                if f is None:
                    continue

                # Lire et parser le XML en mémoire
                tree = ET.parse(f)
                root = tree.getroot()

                print("Racine :", root.tag)
                for child in list(root)[:3]:
                    print(" -", child.tag, child.attrib)


Ouverture du fichier TAR interne : AN_2022001.tar
Lecture du XML : CRI_20220104_001.xml
Racine : PublicationDANBlanc
 - Metadonnees {}
 - ContenuDANBlanc {}
Lecture du XML : AAA_20220104_001.xml
Racine : PublicationDANBleu
 - Metadonnees {}
 - ContenuDANBleu {}
